In [108]:
import os
import datetime
import geopandas as gpd
from shapely.geometry import box
import json

prj = "./main.go"
prj_name = "Amite"
description = "Amite Go-Consequences Model Based on Dewberry Amite River HEC-RAS Model Results"
data_dir = "./data"
out_dir = "./output"
# prj_dir, prj_file_tail = os.path.split(prj)
# prj_name = prj_file_tail.split(".")[0]
with open(prj, "r") as f:
     lines = f.readlines()
lines = [s.strip('\n') for s in lines]
lines

['package main',
 '',
 'import (',
 '\t"fmt"',
 '',
 '\t"github.com/USACE/go-consequences/compute"',
 '\t"github.com/USACE/go-consequences/hazardproviders"',
 '\t"github.com/USACE/go-consequences/resultswriters"',
 '\t"github.com/USACE/go-consequences/structureprovider"',
 ')',
 '',
 'func main() {',
 '\tfmt.Println("Start")',
 '',
 '\t//define flood hazard, provide the file path to your desired flood',
 '\thazard, e := hazardproviders.Init("./data/flood_here.tif")',
 '\tif e != nil {',
 '\t\tpanic(e)',
 '\t}',
 '\tdefer hazard.Close()',
 '',
 '\t//define structure inventory, provide file path to structure inventory',
 '\tinventory, b := structureprovider.InitSHP("./data/inventory_here.shp")',
 '\tif b != nil {',
 '\t\tpanic(b)',
 '\t}',
 '',
 '\t//define result file, provide file path where you want results to be saved',
 '\t// results, c := resultswriters.InitGpkResultsWriter("./output/result_file.gpkg", "event_results")',
 '\tresults, c := resultswriters.InitGpkResultsWriter_Project

In [92]:
# Get Hazard Layer (WSE geotif from HEC-RAS)
# splitting by double quotes because GoLang always uses double quotes for defining strings.
try:
    hazard_layer = [s for s in lines if "hazardproviders.Init" in s][0].split('"')[1]
except IndexError:
    print ("No hazard layer found. Please Specify the Layer Manually. Setting Layer to None.")
    hazard_layer = None

In [93]:
# Get Structure Inventory Layer
try: 
    structure_inventory_layer = [s for s in lines if "structureprovider.Init" in s][0].split('"')[1]
    # Create a bounding box feature from the structure inventory layer
    gdf = gpd.read_file(structure_inventory_layer)
    crs = gdf.crs
    gdf.to_crs(epsg=4326, inplace=True)
    bounding_box = box(*gdf.total_bounds)
    gpd.GeoSeries([bounding_box]).to_file("bounding_box.geojson", driver='GeoJSON')
    spatial_extent = str(gpd.GeoSeries([bounding_box])[0])
except IndexError:
    print ("No structure inventory layer found. Please Specify the Layer Manually. Setting Layer and spatial_extent to None.")
    structure_inventory_layer = None
    crs = None
    spatial_extent = None


In [94]:
# Get Projected Results Layer
try:
    results_layer = [s for s in lines if "resultswriters.InitGpkResultsWriter_Projected" in s][0].split('"')[1]
    # Getting the projection from the last parameter of the InitGpkResultsWriter_Projected function call.
    results_projection = [s for s in lines if "resultswriters.InitGpkResultsWriter_Projected" in s][0].split(',')[-1].split(')')[0]
except:
    # Get Results Layer that assumes 4326 projection
    # If resusltswriters line not found, exception will be thrown and the results layer will be set to None.
    try:
        results_layer = [s for s in lines if "resultswriters.InitGpkResultsWriter" in s][0].split('"')[1]
        results_projection = "4326"
    except IndexError:
        print ("No results layer found. Please Specify the Layer Manually. Setting Layer to None.")
        results_layer = None
        results_projection = None

In [95]:
# Get list of tifs from data directory
tif_endswith = [".tif", ".tiff", ".geotif", ".geotiff"]
tif_list = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.lower().endswith(tuple(tif_endswith))]
tif_list = [i.replace('\\', '/') for i in tif_list]
if len(tif_list) == 0:
    print("No tifs found in data directory. Please Specify the Layer Manually. Setting Common Files Hazard Layer List to None.")
    tif_list = None

# Get list of shps from data directory
shp_endswith = [".shp", ".geojson", ".json", ".gpkg"]
shp_list = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.lower().endswith(tuple(shp_endswith))]
shp_list = [i.replace('\\', '/') for i in shp_list]
if len(shp_list) == 0:
    print("No shps found in data directory. Please Specify the Layer Manually. \
          Setting Common Files Structure Inventory Layer List to None.")
    shp_list = None

# Get list of output files from out_dir.
output_endswith = [".gpkg"]
output_list = [os.path.join(out_dir, f) for f in os.listdir(out_dir) if f.lower().endswith(tuple(output_endswith))]
output_list = [i.replace('\\', '/') for i in output_list]
if len(output_list) == 0:
    print("No gpkg files found in output directory. Please Specify the Layer Manually. \
          Setting Common Files Output Layer List to None.")
    output_list = None

No gpkg files found in output directory. Please Specify the Layer Manually.           Setting Common Files Output Layer List to None.


In [102]:
# Output model_application.json
model_application_template_fn = "C:\py\hec_meta_extract\dev\go-consequences\go_consequences_model_application_template.json"
with open(model_application_template_fn, "r") as f:
    model_application_template = json.load(f)

model_application_template.keys()

dict_keys(['_id', 'spatial_extent', 'common_parameters', 'spatial_valid_extent', 'common_software_version', 'keywords', 'temporal_resolution', 'temporal_extent', 'title', 'authors', 'grid', 'purpose', 'common_files_details', 'spatial_valid_extent_resolved', 'linked_resources', 'spatial_extent_resolved', 'description', 'application_date'])

In [103]:
dropkeys_list = ['_id', 'keywords', 'common_parameters', 'spatial_valid_extent', 'common_software_version', 'temporal_resolution', \
                 'temporal_extent', 'spatial_valid_extent_resolved', 'linked_resources', 'spatial_extent_resolved', \
                 'authors', 'purpose']
key_list = list(model_application_template.keys())
for dropkey in dropkeys_list:
    model_application_template.pop(dropkey)

In [104]:
model_application_template.keys()

dict_keys(['spatial_extent', 'title', 'grid', 'common_files_details', 'description', 'application_date'])

In [105]:
model_application_template['spatial_extent'] = spatial_extent
model_application_template['title'] = f'Go-Consequences {prj_name}'
model_application_template['description'] = f'{description}'
model_application_template['grid']['coordinate_system'] = str(crs)
model_application_template['application_date'] = datetime.datetime.now().strftime("%Y-%m-%d")



In [106]:
model_application_template['common_files_details'] = []
model_application_template['common_files_details'].append(
    [{
        "description": "The project's Main Go-Consequences run script file",
        "location": prj,
        "source_dataset": None,
        "title": "Project File"
    },
    {
        "description": "There may be multiple Water Surface Elevation Rasters in the Go-Consequences data directory.",
        "location": tif_list,
        "source_dataset": "HEC-RAS",
        "title": "Hazard Layers as Water Surface Elevation Raster Files"
    },
    {
        "description": "There may be multiple Structure Inventories in the Go-Consequences data directory.",
        "location": shp_list,
        "source_dataset": None,
        "title": "Structure Inventory Feature Layers"
    },
    {
        "description": "There may be multiple Output Feature Layers.",
        "location": output_list,
        "source_dataset": "Go-Consequences Output",
        "title": "Output Feature Layers"
    }]
)

In [107]:
# output model application json
output_dir = os.path.join(os.getcwd(), "output")
output_model_json = os.path.join(output_dir,f'{prj_name}_model_application.json')
with open(output_model_json, "w") as outfile:
    json.dump(model_application_template, outfile)